### Efficient and Accurate Scene Text detection pipeline (EAST)

C++ implementation is able to rotate bounding boxes, though Python is not.<br>
Also EAST requires all shapes of image to be **multiple of 32**

In [4]:
import os
import time
import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
layerNames = ["feature_fusion/Conv_7/Sigmoid", "feature_fusion/concat_3"]

In [ ]:
net = cv2.dnn.readNet(path)
blob = cv2.dnn.blobfromImage()
